In [1]:
import os

In [2]:
DATA_DIR = os.path.join(os.getcwd(), 'RSNA_COVID_png_512')

TRAIN_DIR = os.path.join(DATA_DIR, 'train')


In [10]:
for classes in os.listdir(TRAIN_DIR):
    len_class = len(os.listdir(os.path.join(TRAIN_DIR, classes)))
    print(f'{classes}의 비율: ', round(len_class / 4842, 2))

Atypical의 비율:  0.08
Indeterminate의 비율:  0.17
Negative의 비율:  0.28
Typical의 비율:  0.47


In [7]:
379 + 839 +  1340 +  2284

4842

# 우리 모델의 문제

avgpool과 fc 레이어가 중첩되었음.  
그렇기 때문에 output의 channel이 1000이 나옴

![img](/home/pung/repo/Pytorch_MLOps/imgs/model.png)



```
self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  
self.fc = nn.Linear(512 * 4, num_classes)
```

In [11]:
backbone = models.resnet50(pretrained='imagenet')
#self.backbone.conv1 = nn.Conv2d(img_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

checkpoint = torch.load("/home/ubuntu/.cache/torch/checkpoints/resnet50-19c8e357.pth")
conv1_weight = checkpoint['conv1.weight']
# checkpoint['conv1.weight'] = conv1_weight.mean(dim=1, keepdim=True)
print(torch.nn.Parameter(conv1_weight.mean(dim=1, keepdim=True)))

Parameter containing:
tensor([[[[-7.8093e-03,  6.8120e-03,  5.7406e-03,  ...,  1.2872e-02,
            9.1395e-03, -2.8071e-02],
          [-4.5682e-03,  9.6330e-03,  4.4220e-02,  ...,  9.5410e-02,
            8.1550e-02,  6.4484e-02],
          [-2.9053e-02, -5.1312e-02, -5.7291e-02,  ...,  1.1006e-01,
            1.2317e-01,  1.2586e-01],
          ...,
          [ 1.7908e-02,  1.7311e-02, -5.6169e-02,  ..., -2.7396e-01,
           -2.1190e-01, -8.4858e-02],
          [ 5.3169e-02,  9.9450e-02,  1.1513e-01,  ...,  4.2284e-03,
           -9.4499e-02, -8.9947e-02],
          [-3.1243e-02,  3.7058e-02,  8.1318e-02,  ...,  1.3679e-01,
            8.2674e-02,  1.6161e-02]]],


        [[[ 4.0948e-03,  4.3650e-03, -4.6187e-04,  ...,  8.4505e-03,
           -1.6375e-03,  6.1063e-03],
          [ 9.1770e-03,  4.2219e-03,  3.6380e-03,  ...,  5.0033e-03,
            2.8041e-03,  1.9822e-03],
          [-4.6230e-03,  2.1304e-03,  2.8272e-03,  ...,  1.7730e-03,
           -1.8185e-03,  1.4826e-0

In [10]:
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torchvision import models
from torch.utils.model_zoo import load_url as load_state_dict_from_url
import torch 

model_urls = {
    "resnet50": 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
}

class Pre_Resnet50(nn.Module):
    def __init__(self, img_channel, num_classes, pretrained=True):
        super(Pre_Resnet50, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone.conv1 = nn.Conv2d(img_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        checkpoint = torch.load("/home/ubuntu/.cache/torch/checkpoints/resnet50-19c8e357.pth")
        conv1_weight = checkpoint['conv1.weight']
        self.backbone.conv1.weight = torch.nn.Parameter(conv1_weight.mean(dim=1, keepdim=True))

        self.backbone.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.backbone.fc = nn.Linear(512 * 4, num_classes)
    
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.backbone.fc(x)
        return x

if __name__ == '__main__':
    net = Pre_Resnet50(img_channel=1, num_classes=4)
    print(net)


Pre_Resnet50(
  (backbone): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

# 수정사항

In [12]:
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torchvision import models

class Pre_Resnet50(nn.Module):
    def __init__(self, img_channel, num_classes, pretrained=True):
        super(Pre_Resnet50, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone.conv1 = nn.Conv2d(img_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        ##### 변경 #########
        self.backbone.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.backbone.fc = nn.Linear(512 * 4, num_classes)
    
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.backbone.fc(x)
        return x

if __name__ == '__main__':
    net = Pre_Resnet50(img_channel=1, num_classes=4)
    print(net)

Pre_Resnet50(
  (backbone): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    